<a href="https://colab.research.google.com/github/LathaReddybattula/HDS5210_InClass-Latha/blob/master/midterm/midterm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Mid-term for HDS5210

Your supervisor is concerned about 4-year survival risks for COPD. She has asked for you to do some analysis using a new metric, BODE. BODE is an improvement on a previous metric and promises to provide insight on survival risks.

BODE is defined here. https://www.mdcalc.com/calc/3916/bode-index-copd-survival#evidence

Your assignment is to create a BODE calculation, use it to calculate BODE scores and BODE survival rates for a group of patients. Then we want to evaluate the average BODE scores and BODE survival rates for each area hospital.

Your patient input file will have the following columns:
NAME,SSN,LANGUAGE,JOB,HEIGHT_M,WEIGHT_KG,fev_pct,dyspnea_description,distance_in_meters,hospital

BODE calculations require a BMI value, so you will have to create a function for it.

Your output should be in the form of two CSV files, patient_output.csv and hospital_output.csv.

Patient_output will have the following columns:
NAME,BODE_SCORE,BODE_RISK,HOSPITAL

Hospital output will have the following columns:
HOSPITAL_NAME, COPD_COUNT, PCT_OF_COPD_CASES_OVER_BEDS, AVG_SCORE, AVG_RISK

Each function you create should have documentation and a suitable number of test cases. If the input data could be wrong, make sure to raise a Value Error.

For this assignment, use the doctest, json, and csv libraries. Pandas is not allowed for this assignment.

In [32]:
import doctest
import json
import csv

### Step 1: Calculate BMI

In [38]:
def calculate_bmi(weight_kg, height_m):
    """
    Calculate BMI using weight and height.
    >>> calculate_bmi(70, 1.75)
    22.86
    >>> calculate_bmi(100, 1.50)
    44.44
    """
    if weight_kg <= 0 or height_m <= 0:
        raise ValueError("Weight and height must be positive values.")
    return round(weight_kg / (height_m ** 2), 2)


### Step 2: Calculate BODE Score

In [40]:
def calculate_bode(fev_pct, dyspnea_description, distance_in_meters, bmi):
    """
    Calculate BODE score based on FEV percentage, dyspnea description, walk distance, and BMI.
    """
    # Scores for FEV1
    if fev_pct >= 65:
        fev_score = 0
    elif 50 <= fev_pct < 65:
        fev_score = 1
    elif 36 <= fev_pct < 50:
        fev_score = 2
    else:
        fev_score = 3
    # Scores for dyspnea
    dyspnea_map = {
        "None": 0,
        "Mild": 1,
        "Moderate": 2,
        "Severe": 3
    }
    dyspnea_score = dyspnea_map.get(dyspnea_description, 0)
    # Scores for walk distance
    if distance_in_meters >= 350:
        walk_score = 0
    elif 250 <= distance_in_meters < 350:
        walk_score = 1
    elif 150 <= distance_in_meters < 250:
        walk_score = 2
    else:
        walk_score = 3

    # Scores for BMI
    bmi_score = 0 if bmi > 21 else 1
    bode_score = fev_score + dyspnea_score + walk_score + bmi_score
    return bode_score

### Step 3: Calculate BODE Risk

In [43]:
def bode_survival_rate(bode_score):
    """
    Map BODE score to 4-year survival risk.
    """
    survival_rate_map = {
        0: 80, 1: 80, 2: 80,
        3: 67, 4: 67,
        5: 57, 6: 57,
        7: 35, 8: 35,
        9: 18, 10: 18
    }
    return survival_rate_map.get(bode_score, "Unknown")


### Step 4: Load Hospital Data

In [45]:
import csv

def load_hospital_data(filename='hospital_data.csv'):
    """
    Load hospital data from a CSV file and return it as a list of dictionaries.
    Each dictionary represents a hospital with its name and number of beds.

    >>> load_hospital_data('sample_hospital_data.csv')
    [{'HOSPITAL_NAME': 'Hospital A', 'BEDS': 100}, {'HOSPITAL_NAME': 'Hospital B', 'BEDS': 150}]
    """
    hospitals = []
    try:
        with open(filename, mode='r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                hospital_name = row['HOSPITAL_NAME']
                beds = int(row['BEDS'])
                hospitals.append({
                    'HOSPITAL_NAME': hospital_name,
                    'BEDS': beds
                })
    except FileNotFoundError:
        raise FileNotFoundError(f"File {filename} not found.")
    except KeyError as e:
        raise ValueError(f"Missing required column in data: {e}")

    return hospitals


### Step 5: Main business logic

Call BODE Score, BODE Risk functions for each patient.

For each hospital, calculate Avg BODE score and Avg BODE risk and count the number of cases for each hospital.

In [53]:
patient_csv = "patient.csv"
hospital_json = "hospitals.json"

patient_output_file = "patient_output.csv"
hospital_output_file = "hospital_output.csv"

###
import csv
import json
# Step 1: Define file paths
patient_csv = "/content/patient.csv"
hospital_json = "/content/hospitals (1).json"  # Adjusted path
# Step 2: Define a function to calculate the BODE Score and Risk
def calculate_bode_score(patient):
    """
    Calculate BODE score and risk based on patient data.
    You should implement the correct BODE score calculation here.
    """
    bode_score = (patient['HEIGHT_M'] + patient['WEIGHT_KG']) / 2  # Replace this with the real calculation
    bode_risk = bode_score / 10  # This should map to actual survival risk
    return bode_score, bode_risk
# Step 3: Load the patient data from CSV
def load_patient_data(filename):
    patient_data = []
    with open(filename, mode='r') as file:
        csv_reader = csv.DictReader(file)

        # Print the headers (column names)
        headers = csv_reader.fieldnames
        print("CSV Headers:", headers)

        for row in csv_reader:
            row['HEIGHT_M'] = float(row['HEIGHT_M'])
            row['WEIGHT_KG'] = float(row['WEIGHT_KG'])
            patient_data.append(row)

    return patient_data
# Step 4: Load the hospital data from JSON
def load_hospital_data(filename):
    with open(filename, mode='r') as file:
        hospital_data = json.load(file)
    return hospital_data
# Step 5: Process the patients and calculate BODE Scores and Risks
def process_patients_and_hospitals(patient_data, hospital_data):
    patient_results = []
    hospital_aggregates = {}
  # Step 6: Process each patient
    for patient in patient_data:
        # Calculate the BODE score and risk for each patient
        bode_score, bode_risk = calculate_bode_score(patient)
        patient_id = patient['NAME']
        hospital_id = patient['hospital']
        # Store patient result
        patient_results.append([patient_id, hospital_id, bode_score, bode_risk])
        # Aggregate data by hospital
        if hospital_id not in hospital_aggregates:
            hospital_aggregates[hospital_id] = {
                'total_bode_score': 0,
                'total_bode_risk': 0,
                'num_patients': 0
            }
        hospital_aggregates[hospital_id]['total_bode_score'] += bode_score
        hospital_aggregates[hospital_id]['total_bode_risk'] += bode_risk
        hospital_aggregates[hospital_id]['num_patients'] += 1

    # Step 7: Calculate the averages for each hospital
    hospital_output_list = []
    for hospital_id, aggregates in hospital_aggregates.items():
        avg_bode_score = aggregates['total_bode_score'] / aggregates['num_patients']
        avg_bode_risk = aggregates['total_bode_risk'] / aggregates['num_patients']
        hospital_output_list.append([hospital_id, avg_bode_score, avg_bode_risk, aggregates['num_patients']])

    return patient_results, hospital_output_list

# Step 8: Write the results to CSV files
def write_csv(filename, data, headers=None):
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        if headers:
            writer.writerow(headers)
        writer.writerows(data)

# Step 9: Load data, process it, and save the results
patient_data = load_patient_data(patient_csv)
hospital_data = load_hospital_data(hospital_json)

# Process the data and get the results
patient_results, hospital_output_list = process_patients_and_hospitals(patient_data, hospital_data)

# Write the patient and hospital results to their respective CSV files
write_csv("patient_output.csv", patient_results, headers=["PATIENT_NAME", "HOSPITAL", "BODE_SCORE", "BODE_RISK"])
write_csv("hospital_output.csv", hospital_output_list, headers=["HOSPITAL", "AVG_BODE_SCORE", "AVG_BODE_RISK", "NUM_PATIENTS"])

# Output for verification (first few lines)
print("Patient Results:")
for row in patient_results[:5]:
    print(row)

print("\nHospital Results:")
for row in hospital_output_list[:5]:
    print(row)
###
patient_results = []
hospital_output_list = []

#Write Patient_output.csv
with open(patient_output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(patient_results)
#Write Hospital_output.csv
with open(hospital_output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(hospital_output_list)

CSV Headers: ['NAME', 'SSN', 'LANGUAGE', 'JOB', 'HEIGHT_M', 'WEIGHT_KG', 'fev_pct', 'dyspnea_description', 'distance_in_meters', 'hospital']
Patient Results:
['Vanessa Roberts', "ST.LUKE'S", 46.0, 4.6]
['Christopher Fox', 'SAINT LOUIS UNIVERSITY', 42.365, 4.2365]
['Benjamin Johnston', 'BJC', 48.26, 4.826]
['Christopher Hernandez', 'MISSOURI BAPTIST', 41.61, 4.161]
['Valerie Burch', 'BJC WEST COUNTY', 43.144999999999996, 4.3145]

Hospital Results:
["ST.LUKE'S", 49.28707317073169, 4.92870731707317, 164]
['SAINT LOUIS UNIVERSITY', 49.36060975609756, 4.936060975609758, 164]
['BJC', 49.58717391304347, 4.95871739130435, 184]
['MISSOURI BAPTIST', 49.856801242236, 4.985680124223601, 161]
['BJC WEST COUNTY', 49.26999999999999, 4.9270000000000005, 171]
